## Import Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle

#Import function to get the covariate matrix that includes spike history from previous bins
from preprocessing_funcs import get_spikes_with_history

#Import decoder functions
import Decoder_funcs
from Decoder_funcs import get_vaf

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.
Using gpu device 3: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)
/opt/anaconda/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


## Import Data

**Neural data should be a matrix of size "number of time bins" x "number of neurons", where each entry is the firing rate of a given neuron in a given time bin**

**The output you are decoding should be a matrix of size "number of time bins" x "number of features you are decoding"**

In [2]:
# folder='/Users/jig289/Dropbox/MATLAB/Projects/In_Progress/BMI/Processed_Data/'
folder='/home/jglaser2/Data/DecData/'

with open(folder+'test_data.pickle','rb') as f:
    neural_data,vels_binned=pickle.load(f)

## Define Covariates ##

### User Options

In [3]:
bins_current=1 #Whether to use concurrent time bin of neural data
bins_before=13 #How many bins of neural data prior to the output are used for decoding
bins_after=0 #How many bins of neural data after (and including) the output are used for decoding
bins_surrounding=bins_before+bins_before+bins_after

### Get Covariates

In [4]:
#Function to get the covariate matrix that includes spike history from previous bins
X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

#Put in "flat" format for XGB and linear
X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

#Set decoding output
y=vels_binned

### Process Covariates

In [5]:
# zscore = lambda x: (x - scipy.stats.nanmean(x)) / scipy.stats.nanstd(x)
def zscore_nan(X,axis):
    X_zscore=(X - np.nanmean(X,axis=axis)) / np.nanstd(X,axis=axis)
    return X_zscore


In [6]:
#Normalize

X=zscore_nan(X,axis=0)
X_flat=zscore_nan(X_flat,axis=0)

#Zero-center outputs
y_mean=np.mean(y,axis=0)
y=y-y_mean
# y=stats.zscore(y,axis=0)

## Split into training/testing/validation sets

### Remove validation data

In [7]:
#What part of the data you're using (remove the validation set that we fit parameters on)
use_range=[0, 0.85]
num_examples_orig=X.shape[0]
#Below starts at "bins_before" and ends with a boundary of "bins_after"
use_set=np.arange(np.int(np.round(use_range[0]*num_examples_orig)+bins_before),np.int(np.round(use_range[1]*num_examples_orig)-bins_after))
X=X[use_set,:,:]
X_flat=X_flat[use_set,:]
y=y[use_set,:]

In [18]:
num_cv=10
cv_frac=num_cv**-1
num_examples=X.shape[0]
# testing_sets=[] #List of all the testing sets
# training_sets=[] #List of all the training sets

mean_vafs_lin=np.empty(num_cv)
mean_vafs_wc=np.empty(num_cv)
mean_vafs_xgb=np.empty(num_cv)
mean_vafs_dnn=np.empty(num_cv)
mean_vafs_rnn=np.empty(num_cv)
mean_vafs_gru=np.empty(num_cv)
mean_vafs_lstm=np.empty(num_cv)

for i in range(1):#(num_cv):
    testing_range=[i*cv_frac,(i+1)*cv_frac]
    testing_set=np.arange(np.int(np.round(testing_range[0]*num_examples)),np.int(np.round(testing_range[1]*num_examples)))
    training_set=np.arange(num_examples) #Initialize training set as all examples
    training_set=np.delete(training_set,testing_set) #Remove testing set from all examples to create training set
    #Now make sure there's no overlap between the training/testing sets by removing border entries from test set:
    rmv_before=np.arange(0,bins_before)
    testing_set=np.delete(testing_set,rmv_before)
    rmv_after=np.arange(testing_set.shape[0]-bins_after,testing_set.shape[0])
    testing_set=np.delete(testing_set,rmv_after)
    
    ###Split Data###
    X_train=X[training_set,:,:]
    X_flat_train=X_flat[training_set,:]
    y_train=y[training_set,:]

    X_test=X[testing_set,:,:]
    X_flat_test=X_flat[testing_set,:]
    y_test=y[testing_set,:]
    
    
    ##### DECODERS ########
    
    ###WIENER FILTER###
    model_regr=Decoder_funcs.lin_reg_model(X_flat_train,y_train)
    y_test_pred_lin=model_regr.predict(X_flat_test)
    vafs_lin=get_vaf(y_test,y_test_pred_lin)
    mean_vafs_lin[i]=np.mean(vafs_lin)
 
    ###WIENER CASCADE###
    models_wc=Decoder_funcs.wiener_casc_model(X_flat_train,y_train,deg=3)
    y_test_pred_wc=Decoder_funcs.wiener_casc_predict(models_wc,X_flat_test)
    vafs_wc=get_vaf(y_test,y_test_pred_wc)
    mean_vafs_wc[i]=np.mean(vafs_wc)

    ###XGBOOST###    
    model_xgb=Decoder_funcs.xgb_model(X_flat_train,y_train,max_depth=3,num_round=200) #Fit model
    y_test_pred_xgb=Decoder_funcs.xgb_predict(model_xgb,X_flat_test) #Get predictions
    vafs_xgb=get_vaf(y_test,y_test_pred_xgb)
    mean_vafs_xgb[i]=np.mean(vafs_xgb)    
     
    ###DENSE NEURAL NETWORK###    
    model_dnn=Decoder_funcs.dnn_model(X_flat_train,y_train,num_layers=2,units=400,dropout=0,num_epochs=10)
    y_test_pred_dnn=model_dnn.predict(X_test)
    vafs_dnn=get_vaf(y_test,y_test_pred_dnn)
    mean_vafs_dnn[i]=np.mean(vafs_dnn)    
    
    ###SIMPLE RNN###
    model_rnn=Decoder_funcs.SimpleRNN_model(X_train,y_train,units=400,dropout=0,num_epochs=10)
    y_test_pred_rnn=model_rnn.predict(X_test)
    vafs_rnn=get_vaf(y_test,y_test_pred_rnn)
    mean_vafs_rnn[i]=np.mean(vafs_rnn)

    ###GRU###
    model_gru=Decoder_funcs.GRU_model(X_train,y_train,units=400,dropout=0,num_epochs=10)
    y_test_pred_gru=model_gru.predict(X_test)
    vafs_gru=get_vaf(y_test,y_test_pred_gru)
    mean_vafs_gru[i]=np.mean(vafs_gru)
    
    ###LSTM###
    model_lstm=Decoder_funcs.LSTM_model(X_train,y_train,units=400,dropout=0,num_epochs=10)
    y_test_pred_lstm=model_lstm.predict(X_test)
    vafs_lstm=get_vaf(y_test,y_test_pred_lstm)
    mean_vafs_lstm[i]=np.mean(vafs_lstm)

In [17]:
print(np.mean(mean_vafs_lin))
print(np.std(mean_vafs_lin)/np.sqrt(num_cv))

0.78245607358
0.00974913041087


In [25]:
mean_vafs_lin[0]

0.8053751086088532

In [20]:
mean_vafs_xgb

array([ 0.81200129,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [21]:
mean_vafs_rnn

array([  8.44479912e-001,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         6.93977503e-310])

In [22]:
mean_vafs_lstm

array([  8.94592305e-001,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         6.93977503e-310])